In [ ]:
# default_exp conversion

# conversion

> This module defines typecasting and conversion functions for easily manipulating and generating data. The most powerful methods are `cast` and `convert`. Other utilities include `RNG` and `fill`, as well as extensions of binary logic to continuous variables such as `XOR`, and encodings such as gray code.

In [ ]:
#hide
from nbdev import *
from nbdev.imports import *
from nbdev.export import *
from nbdev.sync import *
from nbdev.showdoc import *

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import warnings
with warnings.catch_warnings(): #ignore warnings
    warnings.simplefilter("ignore")
    import typing
    import numpy as np
    from typing import Optional, Tuple, Dict, Callable, Union, Mapping, Sequence, Iterable, List
    from functools import partial
    import warnings

## Type casting

In order to build the modular typecasting callable `cast`, we start by outlining the data types and default conversion behavior of python, before extending it to wrap non-iterable objects in the desired iterable, and convert the elements of iterable objects to the desired non-iterable type (shown by examples).

In [ ]:
#exports
data = Union[None,int,float,list,tuple,str,dict,set,np.ndarray]

In [ ]:
#export
def trycast(obj : data, to : type) -> data:
    '''
    Attempts to typecast `obj` to datatype `to`
    using default type conversion. Fallback
    of more complex casting cases in this module.
    '''
    try:
        return to(obj)
    except:
        return typing.cast(to,obj)

In [ ]:
show_doc(trycast)

<h4 id="trycast" class="doc_header"><code>trycast</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>trycast</code>(**`obj`**:`Union`\[`NoneType`, `int`, `float`, `list`, `tuple`, `str`, `dict`, `set`, `ndarray`\], **`to`**:`type`)

Attempts to typecast `obj` to datatype `to`
using default type conversion. Fallback
of more complex casting cases in this module.

In [ ]:
trycast(0.5,int) #works like int(0.5)

0

In [ ]:
trycast(0,list) #just return 0 as list(0) gives an error - we want an actual list!

0

The default typecasting, as in the above, has some obvious limitations which can be extended with a basic consideration of whether the object is to be treated as an iterable or a non-iterable. To that end,

In [ ]:
#export
def nonitr2itr(obj : data, to : type) -> data:
    '''
    Wrap the noniterable `obj` into the iterable type `to`.
    '''
    if to is list:
        return [obj]
    elif to is tuple:
        return (obj,)
    elif to is str:
        return f'{obj}'
    elif to is dict:
        return {obj:obj}
    elif to is set:
        return {obj}
    elif to is np.ndarray:
        return np.array(obj)
    else:
        return trycast(obj,to)

In [ ]:
show_doc(nonitr2itr)

<h4 id="nonitr2itr" class="doc_header"><code>nonitr2itr</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>nonitr2itr</code>(**`obj`**:`Union`\[`NoneType`, `int`, `float`, `list`, `tuple`, `str`, `dict`, `set`, `ndarray`\], **`to`**:`type`)

Wrap the noniterable `obj` into the iterable type `to`.

In [ ]:
nonitr2itr(0,list)

[0]

In [ ]:
#export
def itr2nonitr(obj : data, to : type) -> data:
    '''
    Treat each element of the iterable `obj` as the 
    noniterable type `to`.
    '''
    t=type(obj)
    if t is dict:
        return {k:trycast(v,to) for k,v in obj.items()}
    else:
        return trycast([trycast(i,to) for i in obj],to)

In [ ]:
show_doc(itr2nonitr)

<h4 id="itr2nonitr" class="doc_header"><code>itr2nonitr</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>itr2nonitr</code>(**`obj`**:`Union`\[`NoneType`, `int`, `float`, `list`, `tuple`, `str`, `dict`, `set`, `ndarray`\], **`to`**:`type`)

Treat each element of the iterable `obj` as the 
noniterable type `to`.

In [ ]:
itr2nonitr([0.5,1.5],int)

[0, 1]

In [ ]:
#export
def itr2itr(obj : data, to : type) -> data:
    '''
    Treat the iterable `obj` as a new iterable of type `to`.
    Treats dictionaries as their items rather than default keys,
    and treats np.ndarray as the callable np.array(obj).
    '''
    if type(obj) is dict:
        return trycast(obj.items(),to)
    elif to is dict:
        return {i:o for i,o in enumerate(obj)}
    elif to is np.ndarray:
        return np.array(obj)
    else:
        return trycast(obj,to)

In [ ]:
itr2itr({0:1},list) #simply treats dict -> itrtype as dict.items() -> itrtype, as list(dict) returns dict.keys()

[(0, 1)]

In [ ]:
itr2itr([0,1],np.ndarray) #makes sure the datatype np.ndarray yields the callable np.array()

array([0, 1])

Now, we combine these conversion cases into a single object, the `Caster` class, which instantiates as a callable dictionary of partially evaluated functions, that typecast according to the type of the object they are called on, and the type to be casted to. The ruleset is stored as a dictionary which can be modified in place to treat objects differently as needed.

In [ ]:
#export
isiter = lambda t: hasattr(t,'__iter__')

class Caster:
    '''
    Universal typecasting class with customizable ruleset.
    Stores the ruleset as a dictionary of dictionaries.
    The outer dictionary stores the type of the object to be converted.
    The inner dictionary stores all the types to convert into.
    The values are partially evaluated functions on the inner type,
    which get called by a class object and evaluated on the outer type.
    The ruleset can be updated and changed as needed.
    '''
    types=[None,int,float,list,tuple,str,dict,set,np.ndarray]
    
    iterables=[t for t in types if isiter(t)]
    
    iterables.remove(str) #want to wrap strings like numbers
                
    def get_rules(types=types,
                  iterables=iterables,
                  itr2itr=itr2itr,
                  nonitr2itr=nonitr2itr,
                  itr2nonitr=itr2nonitr):
        
        rules={t1:{t2:None for t2 in types} for t1 in types}
        noniterables=[t for t in types if t not in iterables]
        Caster.noniterables=noniterables
        for t1 in types:
            for t2 in types:
                if t1 in noniterables and t2 in iterables:
                    rules[t1][t2]=partial(nonitr2itr,to=t2)
                elif t1 in iterables and t2 in noniterables:
                    rules[t1][t2]=partial(itr2nonitr,to=t2)
                elif t1 in iterables and t2 in iterables:
                    rules[t1][t2]=partial(itr2itr,to=t2)
                else:
                    rules[t1][t2]=partial(trycast,to=t2)
                    
        return rules
        
    
    def __init__(self,
                 rules : Optional[dict] = None):
        if rules is None:
            rules=Caster.get_rules()
        self.rules=rules
        
    def __getitem__(self,item):
        return self.rules[item]
    
    
    def __call__(self,
                 obj : data,
                 *args : type):
        res=obj
        for arg in args:
            t=type(res)
            try:
                res=self.rules[t][arg](res)
            except:
                res=trycast(res,arg)
        return res

In [ ]:
show_doc(Caster)

<h2 id="Caster" class="doc_header"><code>class</code> <code>Caster</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>Caster</code>(**`rules`**:`Optional`\[`dict`\]=*`None`*)

Universal typecasting class with customizable ruleset.
Stores the ruleset as a dictionary of dictionaries.
The outer dictionary stores the type of the object to be converted.
The inner dictionary stores all the types to convert into.
The values are partially evaluated functions on the inner type,
which get called by a class object and evaluated on the outer type.
The ruleset can be updated and changed as needed.

In [ ]:
#exports
cast=Caster()

`cast` wraps non-iterables in iterables:

In [ ]:
cast(0,list)

[0]

`cast` converts the elements of an iterable into the desired non-iterable:

In [ ]:
cast([0.5,1.5],int)

[0, 1]

`cast` accepts multiple arguments for sequential conversion:

In [ ]:
cast([0,1.1,1.5,2],int,set,list) #convert to int, get unique elements, return as list

[0, 1, 2]

By default, we choose to treat strings as non-iterables. This proves useful in later applications of the module.

In [ ]:
cast('hi',list)

['hi']

If we want to change this, we can do so by examining how `cast` converts objects:

In [ ]:
cast[str]

{None: functools.partial(<function trycast at 0x00000273E43FD318>, to=None),
 int: functools.partial(<function trycast at 0x00000273E43FD318>, to=<class 'int'>),
 float: functools.partial(<function trycast at 0x00000273E43FD318>, to=<class 'float'>),
 list: functools.partial(<function nonitr2itr at 0x00000273E51B8828>, to=<class 'list'>),
 tuple: functools.partial(<function nonitr2itr at 0x00000273E51B8828>, to=<class 'tuple'>),
 str: functools.partial(<function trycast at 0x00000273E43FD318>, to=<class 'str'>),
 dict: functools.partial(<function nonitr2itr at 0x00000273E51B8828>, to=<class 'dict'>),
 set: functools.partial(<function nonitr2itr at 0x00000273E51B8828>, to=<class 'set'>),
 numpy.ndarray: functools.partial(<function nonitr2itr at 0x00000273E51B8828>, to=<class 'numpy.ndarray'>)}

It is a nested dictionary, keyed by the datatype to be converted, and valued by an inner dict, which is keyed by the datatype to convert to, and valued with a function converting from the outer key to the inner key. To change how the *instance* evaluates strings to lists for example, 

In [ ]:
cast[str][list] = lambda s: list(s)

In [ ]:
cast('hi',list)

['h', 'i']

If instead we want to change the default behavior of strings to be treated as an iterable, then we can modify the Caster class `iterables`:

In [ ]:
Caster.iterables

[list, tuple, dict, set, numpy.ndarray]

In [ ]:
Caster.iterables.append(str)

Now we re-instantiate a class instance,

In [ ]:
cast=Caster()

In [ ]:
cast('hi',tuple)

('h', 'i')

and strings are treated as iterables, as in native python types. This process can be applied to any datatype, include new ones, etc. We can change it back again by:

In [ ]:
Caster.iterables.remove(str)
cast=Caster()

Now let's see all the types of data conversions, with a parsing function for easy printing of datatypes:

In [ ]:
#export
def typestr(x : data):
    '''
    Parses the string of the input type for readability.
    '''
    if type(x) is type: #if passing type itself
        s=x
    else: #otherwise get type of obj
        s=type(x)
    return str(s).split('<')[-1].split('>')[0].split('class')[-1].split('\'')[1]

In [ ]:
#nonitrs->noniters
for n in [0,1.5]:
    for t in Caster.noniterables:
        print(f'From {n} to {typestr(t)}: {cast(n,t)}')

From 0 to NoneType: 0
From 0 to int: 0
From 0 to float: 0.0
From 0 to str: 0
From 1.5 to NoneType: 1.5
From 1.5 to int: 1
From 1.5 to float: 1.5
From 1.5 to str: 1.5


In [ ]:
#nonitrs->iters
for n in [1.5,'1.5']:
    for t in Caster.iterables:
        print(f'From {n} to {typestr(t)}: {cast(n,t)}')

From 1.5 to list: [1.5]
From 1.5 to tuple: (1.5,)
From 1.5 to dict: {1.5: 1.5}
From 1.5 to set: {1.5}
From 1.5 to numpy.ndarray: 1.5
From 1.5 to list: ['1.5']
From 1.5 to tuple: ('1.5',)
From 1.5 to dict: {'1.5': '1.5'}
From 1.5 to set: {'1.5'}
From 1.5 to numpy.ndarray: 1.5


In [ ]:
#itrs->nonitrs
for n in [[0,0.1],(0,0.1),{0:0.1},{0,0.1},np.array([0,0.1])]:
    for t in Caster.noniterables:
        print(f'From {n} to {typestr(t)}: {cast(n,t)}')

From [0, 0.1] to NoneType: [0, 0.1]
From [0, 0.1] to int: [0, 0]
From [0, 0.1] to float: [0.0, 0.1]
From [0, 0.1] to str: ['0', '0.1']
From (0, 0.1) to NoneType: [0, 0.1]
From (0, 0.1) to int: [0, 0]
From (0, 0.1) to float: [0.0, 0.1]
From (0, 0.1) to str: ['0', '0.1']
From {0: 0.1} to NoneType: {0: 0.1}
From {0: 0.1} to int: {0: 0}
From {0: 0.1} to float: {0: 0.1}
From {0: 0.1} to str: {0: '0.1'}
From {0, 0.1} to NoneType: [0, 0.1]
From {0, 0.1} to int: [0, 0]
From {0, 0.1} to float: [0.0, 0.1]
From {0, 0.1} to str: ['0', '0.1']
From [0.  0.1] to NoneType: [0.0, 0.1]
From [0.  0.1] to int: [0, 0]
From [0.  0.1] to float: [0.0, 0.1]
From [0.  0.1] to str: ['0.0', '0.1']


In [ ]:
#itrs->itrs:
for n in [[0,0.1],(0,0.1),{0:0.1},{0,0.1},np.array([0,0.1])]:
    for t in Caster.iterables:
        print(f'From {n} to {typestr(t)}: {cast(n,t)}')

From [0, 0.1] to list: [0, 0.1]
From [0, 0.1] to tuple: (0, 0.1)
From [0, 0.1] to dict: {0: 0, 1: 0.1}
From [0, 0.1] to set: {0, 0.1}
From [0, 0.1] to numpy.ndarray: [0.  0.1]
From (0, 0.1) to list: [0, 0.1]
From (0, 0.1) to tuple: (0, 0.1)
From (0, 0.1) to dict: {0: 0, 1: 0.1}
From (0, 0.1) to set: {0, 0.1}
From (0, 0.1) to numpy.ndarray: [0.  0.1]
From {0: 0.1} to list: [(0, 0.1)]
From {0: 0.1} to tuple: ((0, 0.1),)
From {0: 0.1} to dict: {0: 0.1}
From {0: 0.1} to set: {(0, 0.1)}
From {0: 0.1} to numpy.ndarray: dict_items([(0, 0.1)])
From {0, 0.1} to list: [0, 0.1]
From {0, 0.1} to tuple: (0, 0.1)
From {0, 0.1} to dict: {0: 0, 1: 0.1}
From {0, 0.1} to set: {0, 0.1}
From {0, 0.1} to numpy.ndarray: {0, 0.1}
From [0.  0.1] to list: [0.0, 0.1]
From [0.  0.1] to tuple: (0.0, 0.1)
From [0.  0.1] to dict: {0: 0.0, 1: 0.1}
From [0.  0.1] to set: {0.0, 0.1}
From [0.  0.1] to numpy.ndarray: [0.  0.1]


In conclusion, `cast` can sequentially transition python objects between iterable and noniterable datatypes or any other datatype, with the fallback of using the default python conversions or simply returning the object, and can be updated as a dictionary of callables to include new typecasting behavior.

## Data Conversions

Here we present useful data type conversion, like decimal numbers to binary arrays, culminating in the `convert` function. We also give various helpers such as `RNG` and analytic continuation of logic such as `XOR`.

In [ ]:
#export
def pad(data : Union[np.ndarray,list],
          bits : Optional[int] = None,
          to : Union[int,float] = int) -> np.ndarray:
    '''
    Pads an array with zeros, up to a length of `bits`.
    '''
    if bits is None:
        bits=0
    else:
        bits=bits-len(data)
    x=[0 for i in range(bits)]+list(data)
    x=np.array(x).astype(to)
    return x

In [ ]:
show_doc(pad)

<h4 id="pad" class="doc_header"><code>pad</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>pad</code>(**`data`**:`Union`\[`ndarray`, `list`\], **`bits`**:`Optional`\[`int`\]=*`None`*, **`to`**:`Union`\[`int`, `float`\]=*`int`*)

Pads an array with zeros, up to a length of `bits`.

In [ ]:
pad(data=[1,0],bits=5,to=float)

array([0., 0., 0., 1., 0.])

In [ ]:
#export
def fill(x : list,fillwith=np.NaN,mask=True):
    '''
    Turn uneven nested lists `x` into arrays `y` substituting
    missing entries using `fillwith` and optionally masking.
    '''
    length = max(map(len, x))
    y=np.array([xi+[fillwith]*(length-len(xi)) for xi in x])
    if mask:
        if np.isfinite(fillwith):
            y=np.ma.masked_equal(y,fillwith)
        else:
            y=np.ma.masked_invalid(y)
    return y

In [ ]:
show_doc(fill)

<h4 id="fill" class="doc_header"><code>fill</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>fill</code>(**`x`**:`list`, **`fillwith`**=*`nan`*, **`mask`**=*`True`*)

Turn uneven nested lists `x` into arrays `y` substituting
missing entries using `fillwith` and optionally masking.

In [ ]:
fill([[1],[1,1,1]],fillwith=0,mask=False)

array([[1, 0, 0],
       [1, 1, 1]])

In [ ]:
fill([[1],[1,1,1]],fillwith=0,mask=True)

masked_array(
  data=[[1, --, --],
        [1, 1, 1]],
  mask=[[False,  True,  True],
        [False, False, False]],
  fill_value=0)

In [ ]:
#export
def nbits(x : Union[int,float,list,np.ndarray]) -> int:
    '''
    Return the number of bits required to represent the input `x`. 
    '''
    t=type(x)
    if (t is int) or (t is float):
        bits=np.ceil(np.log2(x+1)).astype(int) if x!=0 else 1
    elif (t is list) or (t is np.ndarray):
        bits=len(x)
    return bits

In [ ]:
show_doc(nbits)

<h4 id="nbits" class="doc_header"><code>nbits</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>nbits</code>(**`x`**:`Union`\[`int`, `float`, `list`, `ndarray`\])

Return the number of bits required to represent the input `x`. 

In [ ]:
for i in range(11):
    print(f"i={i}, nbits({i})={nbits(i)}")

i=0, nbits(0)=1
i=1, nbits(1)=1
i=2, nbits(2)=2
i=3, nbits(3)=2
i=4, nbits(4)=3
i=5, nbits(5)=3
i=6, nbits(6)=3
i=7, nbits(7)=3
i=8, nbits(8)=4
i=9, nbits(9)=4
i=10, nbits(10)=4


In [ ]:
#export
def num2ar(x: Union[int,float],
            bits : Optional[int] = None,
            to : Union[int,float] = int) -> np.ndarray:
    '''
    Converts decimal number `x` to array `a` zero-padded with `bits`. 
    '''
    bits=bits or nbits(x) #if None, give default
    form='0'+str(bits)+'b'
    binary=format(int(x),form)
    with warnings.catch_warnings(): #ignore numpy deprecation warning
        warnings.simplefilter("ignore")
        a=np.fromstring(binary,'u1')-ord('0')
    return cast(a,to)

In [ ]:
show_doc(num2ar)

<h4 id="num2ar" class="doc_header"><code>num2ar</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>num2ar</code>(**`x`**:`Union`\[`int`, `float`\], **`bits`**:`Optional`\[`int`\]=*`None`*, **`to`**:`Union`\[`int`, `float`\]=*`int`*)

Converts decimal number `x` to array `a` zero-padded with `bits`. 

In [ ]:
for i in range(11):
    print(f"i={i}, num2ar({i})={num2ar(i)}")

i=0, num2ar(0)=[0]
i=1, num2ar(1)=[1]
i=2, num2ar(2)=[1, 0]
i=3, num2ar(3)=[1, 1]
i=4, num2ar(4)=[1, 0, 0]
i=5, num2ar(5)=[1, 0, 1]
i=6, num2ar(6)=[1, 1, 0]
i=7, num2ar(7)=[1, 1, 1]
i=8, num2ar(8)=[1, 0, 0, 0]
i=9, num2ar(9)=[1, 0, 0, 1]
i=10, num2ar(10)=[1, 0, 1, 0]


In [ ]:
num2ar(10,bits=5,to=float)

[0.0, 1.0, 0.0, 1.0, 0.0]

In [ ]:
#export
def ar2num(a : Union[list,np.ndarray],
            to : Union[int,float] = int):
    '''
    Converts array `a` to decimal number `x`.
    '''
    temp=str()
    a=np.array(a).astype(int)
    for c in a:
        temp+=str(c)
    x=int(temp,2)
    return cast(x,to)

In [ ]:
show_doc(ar2num)

<h4 id="ar2num" class="doc_header"><code>ar2num</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>ar2num</code>(**`a`**:`Union`\[`list`, `ndarray`\], **`to`**:`Union`\[`int`, `float`\]=*`int`*)

Converts array `a` to decimal number `x`.

In [ ]:
for i in range(4):
    print(f"a={num2ar(i)}, ar2num(a)={ar2num(num2ar(i))}")

a=[0], ar2num(a)=0
a=[1], ar2num(a)=1
a=[1, 0], ar2num(a)=2
a=[1, 1], ar2num(a)=3


In [ ]:
#export
def ar2hex(a : Union[list,np.ndarray],
            bits : Optional[int] = None,
            prefix : bool = True) -> str:
    '''
    Converts binary array to hex string
    in:
        a (numpy array) : binary array to convert
    out:
        h (str) : hex conversion of a 
    '''
    bits=bits or nbits(a)
    form='0'+str(int(np.log2(bits)))+'x'
    h=format(ar2num(a),form)
    if prefix:
        h='0x'+h
    return h

In [ ]:
show_doc(ar2hex)

<h4 id="ar2hex" class="doc_header"><code>ar2hex</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>ar2hex</code>(**`a`**:`Union`\[`list`, `ndarray`\], **`bits`**:`Optional`\[`int`\]=*`None`*, **`prefix`**:`bool`=*`True`*)

Converts binary array to hex string
in:
    a (numpy array) : binary array to convert
out:
    h (str) : hex conversion of a 

In [ ]:
ar2hex(num2ar(10))

'0x0a'

In [ ]:
ar2hex(num2ar(10),prefix=False)

'0a'

In [ ]:
#export
def hex2ar(h : str,
            bits : Optional[int] = None,
            to : Union[int,float] = int) -> np.ndarray:
    '''
    Converts a hex string `h` into an array `a` padded with `bits` and elements of `astype`.
    '''
    x=int(h,16)
    a=num2ar(x,bits,to)
    return a

In [ ]:
show_doc(hex2ar)

<h4 id="hex2ar" class="doc_header"><code>hex2ar</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>hex2ar</code>(**`h`**:`str`, **`bits`**:`Optional`\[`int`\]=*`None`*, **`to`**:`Union`\[`int`, `float`\]=*`int`*)

Converts a hex string `h` into an array `a` padded with `bits` and elements of `astype`.

In [ ]:
hex2ar('a')

[1, 0, 1, 0]

In [ ]:
hex2ar('0xa')

[1, 0, 1, 0]

In [ ]:
#export
def str2ar(s : str,
            to : Union[list,np.ndarray] = np.ndarray) -> Union[list,np.ndarray]:
    '''
    Converts an input string `s` into an array or list `a` as per `astype`.
    '''
    a=[int(i) for i in s]
    return cast(a,to)

In [ ]:
show_doc(str2ar)

<h4 id="str2ar" class="doc_header"><code>str2ar</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>str2ar</code>(**`s`**:`str`, **`to`**:`Union`\[`list`, `ndarray`\]=*`ndarray`*)

Converts an input string `s` into an array or list `a` as per `astype`.

In [ ]:
str2ar('111')

array([1, 1, 1])

In [ ]:
#export
def ar2str(a : Union[list,np.ndarray], to : data = None) -> str:
    '''
    Converts an input array `a` into a string `s`.
    '''
    s=''.join([str(cast(i,to)) for i in a])
    return s

In [ ]:
show_doc(ar2str)

<h4 id="ar2str" class="doc_header"><code>ar2str</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>ar2str</code>(**`a`**:`Union`\[`list`, `ndarray`\], **`to`**:`Union`\[`NoneType`, `int`, `float`, `list`, `tuple`, `str`, `dict`, `set`, `ndarray`\]=*`None`*)

Converts an input array `a` into a string `s`.

In [ ]:
ar2str([1.5,1.5,1.3],int)

'111'

In [ ]:
#export
def COPY(x : Union[int,float]) -> Union[int,float]:
    '''
    Simply returns `x`. 
    '''
    return x

def NOT(x : Union[int,float]) -> Union[int,float]:
    '''
    Return conjugate of `x`. 
    '''
    return 1-x

def AND(x : Union[int,float],
        y : Union[int,float]) -> Union[int,float]:
    '''
    Return logical AND of `x` and `y`.
    '''
    return x*y

def OR(x : Union[int,float],
       y : Union[int,float]) -> Union[int,float]:
    '''
    Return logical OR of `x` and `y`. See DeMorgan's Laws.
    '''
    return x+y-x*y

def Exclusive_OR(x : Union[int,float],
                 y : Union[int,float]) -> Union[int,float]:
    '''
    Return logical exclusive OR of `x` and `y`. See DeMorgan's Laws.
    '''
    return OR( AND( x , NOT(y) ) , AND ( NOT(x) , y) )

def XOR(*args : Union[int,float,list,np.ndarray]) -> Union[int,float]:
    '''
    Arbitrary input XOR using recursiveness.
    '''
    x=0
    for a in args:
        x=Exclusive_OR(x,a)
    return x

In [ ]:
show_doc(NOT)

<h4 id="NOT" class="doc_header"><code>NOT</code><a href="__main__.py#L8" class="source_link" style="float:right">[source]</a></h4>

> <code>NOT</code>(**`x`**:`Union`\[`int`, `float`\])

Return conjugate of `x`. 

In [ ]:
NOT(1)

0

In [ ]:
NOT(0.25)

0.75

In [ ]:
show_doc(AND)

<h4 id="AND" class="doc_header"><code>AND</code><a href="__main__.py#L14" class="source_link" style="float:right">[source]</a></h4>

> <code>AND</code>(**`x`**:`Union`\[`int`, `float`\], **`y`**:`Union`\[`int`, `float`\])

Return logical AND of `x` and `y`.

In [ ]:
AND(1,1)

1

In [ ]:
AND(1,0.5)

0.5

In [ ]:
show_doc(OR)

<h4 id="OR" class="doc_header"><code>OR</code><a href="__main__.py#L21" class="source_link" style="float:right">[source]</a></h4>

> <code>OR</code>(**`x`**:`Union`\[`int`, `float`\], **`y`**:`Union`\[`int`, `float`\])

Return logical OR of `x` and `y`. See DeMorgan's Laws.

In [ ]:
OR(1,0)

1

In [ ]:
OR(0.5,0.5)

0.75

In [ ]:
show_doc(XOR)

<h4 id="XOR" class="doc_header"><code>XOR</code><a href="__main__.py#L35" class="source_link" style="float:right">[source]</a></h4>

> <code>XOR</code>(**\*`args`**:`Union`\[`int`, `float`, `list`, `ndarray`\])

Arbitrary input XOR using recursiveness.

In [ ]:
XOR(1,1)

0

In [ ]:
XOR(1,1,1)

1

In [ ]:
XOR(0.5,0.5,0.5)

0.4384765625

In [ ]:
for x,y in np.ndindex((2,2)):
    for z in [AND,OR,XOR]:
        print(f"{z.__name__}{x,y}={z(x,y)}")

AND(0, 0)=0
OR(0, 0)=0
XOR(0, 0)=0
AND(0, 1)=0
OR(0, 1)=1
XOR(0, 1)=1
AND(1, 0)=0
OR(1, 0)=1
XOR(1, 0)=1
AND(1, 1)=1
OR(1, 1)=1
XOR(1, 1)=0


In [ ]:
#export
def ar2gr(binary : Union[list,np.ndarray],
             to : Union[list,np.ndarray] = np.ndarray) -> Union[list,np.ndarray]:
    '''
    Converts an input binary array to graycode.
    '''
    binary = cast(binary,int)
    gray = []
    gray += [binary[0]]
    for i in range(1,len(binary)):
        gray += [XOR(binary[i - 1], binary[i])]
    return cast(cast(gray,int),to)

In [ ]:
show_doc(ar2gr)

<h4 id="ar2gr" class="doc_header"><code>ar2gr</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>ar2gr</code>(**`binary`**:`Union`\[`list`, `ndarray`\], **`to`**:`Union`\[`list`, `ndarray`\]=*`ndarray`*)

Converts an input binary array to graycode.

In [ ]:
ar2gr(num2ar(10))

array([1, 1, 1, 1])

In [ ]:
#export
def gr2ar(gray : Union[list,np.ndarray],
             to : Union[list,np.ndarray] = np.ndarray) -> Union[list,np.ndarray]:
    '''
    Converts a gray-code array into binary.
    '''
    binary = []
    binary += [gray[0]]
    for i in range(1, len(gray)):
        if (gray[i] == 0):
            binary += [binary[i - 1]]
        else:
            binary += [NOT(binary[i - 1])]
    return cast(cast(binary,int),to)


In [ ]:
show_doc(gr2ar)

<h4 id="gr2ar" class="doc_header"><code>gr2ar</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>gr2ar</code>(**`gray`**:`Union`\[`list`, `ndarray`\], **`to`**:`Union`\[`list`, `ndarray`\]=*`ndarray`*)

Converts a gray-code array into binary.

In [ ]:
gr2ar(ar2gr(num2ar(10)))

array([1, 0, 1, 0])

In [ ]:
#export
def num2gr(x:int,to:data=None):
    '''
    Converts decimal number `x` to equivalent gray-code number.
    '''
    return int(ar2str(ar2gr(num2ar(x))),2)

In [ ]:
show_doc(num2gr)

<h4 id="num2gr" class="doc_header"><code>num2gr</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>num2gr</code>(**`x`**:`int`, **`to`**:`Union`\[`NoneType`, `int`, `float`, `list`, `tuple`, `str`, `dict`, `set`, `ndarray`\]=*`None`*)

Converts decimal number `x` to equivalent gray-code number.

In [ ]:
for i in range(11):
    print(f"i={i}, num2gr(i)={num2gr(i)}")

i=0, num2gr(i)=0
i=1, num2gr(i)=1
i=2, num2gr(i)=3
i=3, num2gr(i)=2
i=4, num2gr(i)=6
i=5, num2gr(i)=7
i=6, num2gr(i)=5
i=7, num2gr(i)=4
i=8, num2gr(i)=12
i=9, num2gr(i)=13
i=10, num2gr(i)=15


In [ ]:
#export            
def convert(obj : Union[int,float,list,hex,str,np.ndarray],
            to : Union[int,float,list,hex,str,np.ndarray] = np.ndarray,
            bits : Optional[int] = None,
            astype : Union[int,float,list,np.ndarray] = int,
            gray : bool = False):
    '''
    Converts an input `obj` into an output of type `to`, padding with `bits`.
    Internally converts `obj` to np.ndarray with elements of dtype `astype`,
    before converting to the desired dtype `to`. If `gray`, first converts this
    binary array to gray-code. If input or output are `hex`, requires prefix of `0x`.
    
    Possible conversions:
        int -> float
        int -> str
        int -> list 
        int -> array
        int -> hex
        
        str -> int
        str -> float
        str -> list
        str -> array
        str -> hex
        
        list -> arr
        list -> int
        list -> float
        list -> str
        list -> hex
        
        arr -> list
        arr -> int
        arr -> float
        arr -> str
        arr -> hex
        
        hex -> int
        hex -> float
        hex -> arr
        hex -> list
        hex -> str
        
    '''
    
    t=type(obj)
    #first convert to binary numpy array
    if (t is np.ndarray) or (t is list):
        x=obj
    elif (t is int) or (t is float):
        x=num2ar(obj,bits,astype)
    else:# t is str
        if obj[:2]=='0x': #obj is hex
            x=hex2ar(obj[2:],bits,astype) 
        else:
            x=str2ar(obj)
    x=cast(pad(x,bits),astype)
    g=cast(ar2gr(x),astype)
    #convert
    if (to is np.ndarray):
        if gray:
            return g
        else:
            return x
    elif (to is list) or (to is set):
        if gray:
            return to(g)
        else:
            return to(x)
    elif (to is int) or (to is float) or (to is hex):
        if gray:
            return to(ar2num(g))
        else:
            return to(ar2num(x))
    else:# to is str
        if gray:
            return ar2str(g)
        else:
            return ar2str(x)

In [ ]:
show_doc(convert)

<h4 id="convert" class="doc_header"><code>convert</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>convert</code>(**`obj`**:`Union`\[`int`, `float`, `list`, `hex`, `str`, `ndarray`\], **`to`**:`Union`\[`int`, `float`, `list`, `hex`, `str`, `ndarray`\]=*`ndarray`*, **`bits`**:`Optional`\[`int`\]=*`None`*, **`astype`**:`Union`\[`int`, `float`, `list`, `ndarray`\]=*`int`*, **`gray`**:`bool`=*`False`*)

Converts an input `obj` into an output of type `to`, padding with `bits`.
Internally converts `obj` to np.ndarray with elements of dtype `astype`,
before converting to the desired dtype `to`. If `gray`, first converts this
binary array to gray-code. If input or output are `hex`, requires prefix of `0x`.

Possible conversions:
    int -> float
    int -> str
    int -> list 
    int -> array
    int -> hex
    
    str -> int
    str -> float
    str -> list
    str -> array
    str -> hex
    
    list -> arr
    list -> int
    list -> float
    list -> str
    list -> hex
    
    arr -> list
    arr -> int
    arr -> float
    arr -> str
    arr -> hex
    
    hex -> int
    hex -> float
    hex -> arr
    hex -> list
    hex -> str
    

In [ ]:
for i in [10,10.0,'10','0xa']:
    t_in=str(type(i)).split('<')[-1].split('>')[0].split('class')[-1].split('\'')[1]
    for j in [int,float,list,str,hex]:#,set,dict]:
        t_out='hex' if j==hex else str(j).split('<')[-1].split('>')[0].split('class')[-1].split('\'')[1]
        print(f'From {t_in} to {t_out}: convert({i},{t_out})={convert(i,j)}')

From int to int: convert(10,int)=10
From int to float: convert(10,float)=10.0
From int to list: convert(10,list)=[1, 0, 1, 0]
From int to str: convert(10,str)=1010
From int to hex: convert(10,hex)=0xa
From float to int: convert(10.0,int)=10
From float to float: convert(10.0,float)=10.0
From float to list: convert(10.0,list)=[1, 0, 1, 0]
From float to str: convert(10.0,str)=1010
From float to hex: convert(10.0,hex)=0xa
From str to int: convert(10,int)=2
From str to float: convert(10,float)=2.0
From str to list: convert(10,list)=[1, 0]
From str to str: convert(10,str)=10
From str to hex: convert(10,hex)=0x2
From str to int: convert(0xa,int)=10
From str to float: convert(0xa,float)=10.0
From str to list: convert(0xa,list)=[1, 0, 1, 0]
From str to str: convert(0xa,str)=1010
From str to hex: convert(0xa,hex)=0xa


In [ ]:
for i in [10,10.0,'10','0xa']:#,{0,10},{0:10,'a':11}]:
    t_in=str(type(i)).split('<')[-1].split('>')[0].split('class')[-1].split('\'')[1]
    for j in [int,float,list,str,hex]:#,set,dict]:
        t_out='hex' if j==hex else str(j).split('<')[-1].split('>')[0].split('class')[-1].split('\'')[1]
        print(f'Gray Code: From {t_in} to {t_out}: convert({i},{t_out})={convert(i,j,gray=True)}')

Gray Code: From int to int: convert(10,int)=15
Gray Code: From int to float: convert(10,float)=15.0
Gray Code: From int to list: convert(10,list)=[1, 1, 1, 1]
Gray Code: From int to str: convert(10,str)=1111
Gray Code: From int to hex: convert(10,hex)=0xf
Gray Code: From float to int: convert(10.0,int)=15
Gray Code: From float to float: convert(10.0,float)=15.0
Gray Code: From float to list: convert(10.0,list)=[1, 1, 1, 1]
Gray Code: From float to str: convert(10.0,str)=1111
Gray Code: From float to hex: convert(10.0,hex)=0xf
Gray Code: From str to int: convert(10,int)=3
Gray Code: From str to float: convert(10,float)=3.0
Gray Code: From str to list: convert(10,list)=[1, 1]
Gray Code: From str to str: convert(10,str)=11
Gray Code: From str to hex: convert(10,hex)=0x3
Gray Code: From str to int: convert(0xa,int)=15
Gray Code: From str to float: convert(0xa,float)=15.0
Gray Code: From str to list: convert(0xa,list)=[1, 1, 1, 1]
Gray Code: From str to str: convert(0xa,str)=1111
Gray Code:

In [ ]:
#export
def rint(x: Union[int,float,list,np.ndarray]) -> Union[int,np.ndarray]:
    '''
    Typecast rounding to np arrays.
    
    '''
    t=type(x)
    if (t is list) or (t is np.ndarray):
        return np.rint(x).astype(int)
    else:
        return round(x)

In [ ]:
show_doc(rint)

<h4 id="rint" class="doc_header"><code>rint</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>rint</code>(**`x`**:`Union`\[`int`, `float`, `list`, `ndarray`\])

Typecast rounding to np arrays.

In [ ]:
rint(np.array([0.5,0.51]))

array([0, 1])

In [ ]:
#hide
#refresh()
#notebook2script()